In [ ]:
import pandas as pd
import numpy as np
import random
import pickle
import gzip
import re
import fastapy
import gc
from scipy import stats

import plotly     # https://plotly.com/python/getting-started/
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'iframe'
pio.templates.default = "none"
# for image export: conda install python-kaleido==0.1.0

In [ ]:
bases = ['A', 'T', 'G', 'C']
def reverse_complement(dna):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    return ''.join([complement[base] for base in dna[::-1]]) 
def repeat_frames_RC(input_seq):
    return list(pd.Series([''.join(input_seq*2)[i:len(input_seq)+i] for i in range(len(input_seq))] + [reverse_complement(seq) for seq in [''.join(input_seq*2)[i:len(input_seq)+i] for i in range(len(input_seq))]]).sort_values().drop_duplicates())

## Load reference genome
T2T v2.0 Fasta file <br>
download "hs1.fa.gz" from http://hgdownload.soe.ucsc.edu/goldenPath/hs1/bigZips/<br>
also available from https://www.ncbi.nlm.nih.gov/datasets/genome/GCF_009914755.1/<br>
Fasta file can remain compressed<br>

In [ ]:
chrom_list = list(range(1,23))
chrom_list_XY = list(range(1,23)) + ['X', 'Y']

In [ ]:
CHM13_genome_fasta = list(fastapy.parse('genomes/hs1.fa.gz'))
CHM13_genome = dict()
for seq in CHM13_genome_fasta:
    try:
        name = int(seq.id[3:])
    except:
        name = seq.id[3:]
    CHM13_genome[name] = seq.seq.upper()
del CHM13_genome_fasta
del CHM13_genome['M']
gc.collect()

In [ ]:
chrom_lengths = pd.Series([len(CHM13_genome[seq]) for seq in CHM13_genome], index = [seq for seq in CHM13_genome])

In [ ]:
# Function to generate random coordinates matched to any given database of loci
def generate_random_matched_by_length(input_df, reference = CHM13_genome, seq = True, length_col = 'length', chrom_col = 'chrom', start_col = 'start', end_col = 'end'):
    input_df = input_df.copy()
    chrom_lengths = pd.Series([len(reference[seq]) for seq in reference], index = chrom_list_XY)
    if length_col not in input_df.columns:
        input_df[length_col] = input_df[end_col] - input_df[start_col]
    counts_by_chrom = input_df.groupby([chrom_col, length_col])[start_col].count()
    random_coordinates = dict()
    for chrom in set(counts_by_chrom.index.get_level_values(0)):
        print('\r' + str(chrom), end='         ', flush = True)
        random_coordinates[chrom] = dict()
        for length in counts_by_chrom.loc[chrom].index:
            random_coordinates[chrom][length] = pd.DataFrame()
            random_coordinates[chrom][length][start_col] = pd.Series(random.sample(range(chrom_lengths[chrom] - length), counts_by_chrom.loc[chrom][length]))
            random_coordinates[chrom][length][end_col] = random_coordinates[chrom][length][start_col] + length
        random_coordinates[chrom] = pd.concat(random_coordinates[chrom])
    random_coordinates = pd.concat(random_coordinates).reset_index().drop(['level_2'], axis=1)
    random_coordinates.columns = [chrom_col, length_col, start_col, end_col]
    if seq == True:
        random_coordinates['Sequence'] = [reference[chrom][start:end] for chrom, start, end in zip(random_coordinates[chrom_col], random_coordinates[start_col], random_coordinates[end_col])]

    return random_coordinates.sort_values(by = [chrom_col, start_col, end_col]).reset_index(drop = True)

# Counting STRs within other repeat motifs

#### Define some objects

In [ ]:
# regex patterns for finding STRs
str_re = dict()
str_re[1] = re.compile(r'([ATGC]{1}?)\1+'); str_re[2] = re.compile(r'([ATGC]{2}?)\1+'); str_re[3] = re.compile(r'([ATGC]{3}?)\1+'); str_re[4] = re.compile(r'([ATGC]{4}?)\1+'); str_re[5] = re.compile(r'([ATGC]{5}?)\1+'); str_re[6] = re.compile(r'([ATGC]{6}?)\1+'); str_re[7] = re.compile(r'([ATGC]{7}?)\1+'); str_re[8] = re.compile(r'([ATGC]{8}?)\1+'); str_re[9] = re.compile(r'([ATGC]{9,}?)\1+')

In [ ]:
portion_STR = dict(); KDE_all = dict(); vio_range = pd.Series(range(101))/100

# Mirror repeats

### Load MR database

In [ ]:
# Load database of mirror repeats in CHM13 (generated using methods from McGinty and Sunyaev 2023)
MR_all = pd.read_csv('repeat_db/MR.csv.gzip', compression = 'gzip', low_memory = False)
MR_all['chrom'] = [str(chrom) if chrom in ['X', 'Y'] else int(chrom) for chrom in MR_all['chrom']]
MR_all['stem_len'] = MR_all['L_end'] - MR_all['L_start']
MR_all['spacer'] = MR_all['R_start'] - MR_all['L_end']
MR_all['seq_L'] = [CHM13_genome[chrom][start: end+1] for chrom, start, end in zip(MR_all['chrom'], MR_all['L_start'], MR_all['L_end'])]

In [ ]:
# For fully-overlapping entries, keep the one with the longest stem length
# Note: step has already been done for database in Supplemental Files
MR_all = MR_all.sort_values(by = ['chrom', 'L_start', 'stem_len']).drop_duplicates(subset = ['chrom', 'L_start'], keep = 'last')

In [ ]:
MR_all['%R'] = (MR_all['seq_L'].str.count('A') + MR_all['seq_L'].str.count('G') + MR_all['seq_R'].str.count('A') + MR_all['seq_R'].str.count('G')) / (2 * MR_all['stem_len'])
MR_all['%AT'] = (MR_all['seq_L'].str.count('A') + MR_all['seq_L'].str.count('T') + MR_all['seq_R'].str.count('A') + MR_all['seq_R'].str.count('T')) / (2 * MR_all['stem_len'])
MR_all['%AC'] = (MR_all['seq_L'].str.count('A') + MR_all['seq_L'].str.count('C') + MR_all['seq_R'].str.count('A') + MR_all['seq_R'].str.count('C')) / (2 * MR_all['stem_len'])

## Find STRs within MRs

In [ ]:
for n in range(1,10):
    print('\r' + str(n), end = '  ')
    MR_all['STR_'+str(n)] = [list(set(str_re[n].findall(seq))) for seq in MR_all['seq_L']]
MR_all['STR_1-9_seqL'] = MR_all['STR_1'] + MR_all['STR_2'] + MR_all['STR_3'] + MR_all['STR_4'] + MR_all['STR_5'] + MR_all['STR_6'] + MR_all['STR_7'] + MR_all['STR_8'] + MR_all['STR_9']
MR_all['STR_1-9_seqL'] = MR_all['STR_1-9_seqL'].apply(lambda x: list(set(x)))
del MR_all['STR_1']; del MR_all['STR_2']; del MR_all['STR_3']; del MR_all['STR_4']; del MR_all['STR_5']; del MR_all['STR_6']; del MR_all['STR_7']; del MR_all['STR_8']; del MR_all['STR_9']
MR_all['total_nt_STRs_max_seqL'] = [max([len(re.findall(STR, seq)) * len(STR) for STR in STRs], default = 0) for seq, STRs in zip(MR_all['seq_L'], MR_all['STR_1-9_seqL'])]
MR_all['portion_STR_seqL'] = MR_all['total_nt_STRs_max_seqL'] / MR_all['stem_len']
MR_all['diff_STR_seqL'] = MR_all['stem_len'] - MR_all['total_nt_STRs_max_seqL']

In [ ]:
MR_portion = pd.concat([MR_all.groupby(['stem_len'])['portion_STR_seqL'].mean(), MR_all.groupby(['stem_len'])['portion_STR_seqL'].quantile(0.975), MR_all.groupby(['stem_len'])['portion_STR_seqL'].quantile(0.025)], axis=1)
MR_portion.columns = ['mean', 'high', 'low']
portion_STR['MR'] = MR_portion

#### STR content in MRs according to sequence composition

In [ ]:
MR_hRhY = MR_all.loc[(MR_all['%R'] > 0.95) | (MR_all['%R'] < 0.05)].copy()
MR_hRhY_portion = pd.concat([MR_hRhY.groupby(['stem_len'])['portion_STR_seqL'].mean(), MR_hRhY.groupby(['stem_len'])['portion_STR_seqL'].quantile(0.975), MR_hRhY.groupby(['stem_len'])['portion_STR_seqL'].quantile(0.025)], axis=1)
MR_hRhY_portion.columns = ['mean', 'high', 'low']
portion_STR['MR_hRhY'] = MR_hRhY_portion

In [ ]:
MR_other = MR_all.loc[(MR_all['%R'] < 0.95) & (MR_all['%R'] > 0.05) & (MR_all['%AT'] < 0.95) & (MR_all['%AT'] > 0.05) & (MR_all['%AC'] < 0.95) & (MR_all['%AC'] > 0.05)].copy()
MR_other_portion = pd.concat([MR_other.groupby(['stem_len'])['portion_STR_seqL'].mean(), MR_other.groupby(['stem_len'])['portion_STR_seqL'].quantile(0.975), MR_other.groupby(['stem_len'])['portion_STR_seqL'].quantile(0.025)], axis=1)
MR_other_portion.columns = ['mean', 'high', 'low']
portion_STR['MR_other'] = MR_other_portion

### Random matched coordinates

#### Matched for all MRs

In [ ]:
random_matched_MR_all = generate_random_matched_by_length(MR_all[['chrom', 'L_start', 'L_end']], start_col = 'L_start', end_col = 'L_end')

In [ ]:
for n in range(1,10):
    print('\r' + str(n), end = '  ')
    random_matched_MR_all['STR_'+str(n)] = [list(set(str_re[n].findall(seq))) for seq in random_matched_MR_all['Sequence']]
random_matched_MR_all['STR_1-9_seq'] = random_matched_MR_all['STR_1'] + random_matched_MR_all['STR_2'] + random_matched_MR_all['STR_3'] + random_matched_MR_all['STR_4'] + random_matched_MR_all['STR_5'] + random_matched_MR_all['STR_6'] + random_matched_MR_all['STR_7'] + random_matched_MR_all['STR_8'] + random_matched_MR_all['STR_9']
random_matched_MR_all['STR_1-9_seq'] = random_matched_MR_all['STR_1-9_seq'].apply(lambda x: list(set(x)))
del random_matched_MR_all['STR_1']; del random_matched_MR_all['STR_2']; del random_matched_MR_all['STR_3']; del random_matched_MR_all['STR_4']; del random_matched_MR_all['STR_5']; del random_matched_MR_all['STR_6']; del random_matched_MR_all['STR_7']; del random_matched_MR_all['STR_8']; del random_matched_MR_all['STR_9']
random_matched_MR_all['total_nt_STRs_max_seq'] = [max([len(re.findall(STR, seq)) * len(STR) for STR in STRs], default = 0) for seq, STRs in zip(random_matched_MR_all['Sequence'], random_matched_MR_all['STR_1-9_seq'])]
random_matched_MR_all['portion_STR_seq'] = random_matched_MR_all['total_nt_STRs_max_seq'] / random_matched_MR_all['length']
random_matched_MR_all['diff_STR_seq'] = random_matched_MR_all['length'] - random_matched_MR_all['total_nt_STRs_max_seq']

portion_STR['random_MR'] = random_matched_MR_portion

#### Matched for hR/HY MRs, non-hR/hY MRs

In [ ]:
random_matched_MR_hRhY = generate_random_matched_by_length(MR_hRhY[['chrom', 'L_start', 'L_end']], start_col = 'L_start', end_col = 'L_end')

In [ ]:
for n in range(1,10):
    print('\r' + str(n), end = '  ')
    random_matched_MR_hRhY['STR_'+str(n)] = [list(set(str_re[n].findall(seq))) for seq in random_matched_MR_hRhY['Sequence']]
random_matched_MR_hRhY['STR_1-9_seq'] = random_matched_MR_hRhY['STR_1'] + random_matched_MR_hRhY['STR_2'] + random_matched_MR_hRhY['STR_3'] + random_matched_MR_hRhY['STR_4'] + random_matched_MR_hRhY['STR_5'] + random_matched_MR_hRhY['STR_6'] + random_matched_MR_hRhY['STR_7'] + random_matched_MR_hRhY['STR_8'] + random_matched_MR_hRhY['STR_9']
random_matched_MR_hRhY['STR_1-9_seq'] = random_matched_MR_hRhY['STR_1-9_seq'].apply(lambda x: list(set(x)))
del random_matched_MR_hRhY['STR_1']; del random_matched_MR_hRhY['STR_2']; del random_matched_MR_hRhY['STR_3']; del random_matched_MR_hRhY['STR_4']; del random_matched_MR_hRhY['STR_5']; del random_matched_MR_hRhY['STR_6']; del random_matched_MR_hRhY['STR_7']; del random_matched_MR_hRhY['STR_8']; del random_matched_MR_hRhY['STR_9']
random_matched_MR_hRhY['total_nt_STRs_max_seq'] = [max([len(re.findall(STR, seq)) * len(STR) for STR in STRs], default = 0) for seq, STRs in zip(random_matched_MR_hRhY['Sequence'], random_matched_MR_hRhY['STR_1-9_seq'])]
random_matched_MR_hRhY['portion_STR_seq'] = random_matched_MR_hRhY['total_nt_STRs_max_seq'] / random_matched_MR_hRhY['length']
random_matched_MR_hRhY['diff_STR_seq'] = random_matched_MR_hRhY['length'] - random_matched_MR_hRhY['total_nt_STRs_max_seq']

portion_STR['random_MR_hRhY'] = random_matched_MR_hRhY_portion

In [ ]:
random_matched_MR_other = generate_random_matched_by_length(MR_other[['chrom', 'L_start', 'L_end']], start_col = 'L_start', end_col = 'L_end')

In [ ]:
for n in range(1,10):
    print('\r' + str(n), end = '  ')
    random_matched_MR_other['STR_'+str(n)] = [list(set(str_re[n].findall(seq))) for seq in random_matched_MR_other['Sequence']]
random_matched_MR_other['STR_1-9_seq'] = random_matched_MR_other['STR_1'] + random_matched_MR_other['STR_2'] + random_matched_MR_other['STR_3'] + random_matched_MR_other['STR_4'] + random_matched_MR_other['STR_5'] + random_matched_MR_other['STR_6'] + random_matched_MR_other['STR_7'] + random_matched_MR_other['STR_8'] + random_matched_MR_other['STR_9']
random_matched_MR_other['STR_1-9_seq'] = random_matched_MR_other['STR_1-9_seq'].apply(lambda x: list(set(x)))
del random_matched_MR_other['STR_1']; del random_matched_MR_other['STR_2']; del random_matched_MR_other['STR_3']; del random_matched_MR_other['STR_4']; del random_matched_MR_other['STR_5']; del random_matched_MR_other['STR_6']; del random_matched_MR_other['STR_7']; del random_matched_MR_other['STR_8']; del random_matched_MR_other['STR_9']
random_matched_MR_other['total_nt_STRs_max_seq'] = [max([len(re.findall(STR, seq)) * len(STR) for STR in STRs], default = 0) for seq, STRs in zip(random_matched_MR_other['Sequence'], random_matched_MR_other['STR_1-9_seq'])]
random_matched_MR_other['portion_STR_seq'] = random_matched_MR_other['total_nt_STRs_max_seq'] / random_matched_MR_other['length']
random_matched_MR_other['diff_STR_seq'] = random_matched_MR_other['length'] - random_matched_MR_other['total_nt_STRs_max_seq']

portion_STR['random_MR_other'] = random_matched_MR_other_portion

### STR probability density

In [ ]:
MR_all['stem_len/10'] = MR_all['stem_len']//10

MR_vio = dict()
for group in range(1,10):
    print('\r' + str(group), end = '    ')
    current = stats.gaussian_kde(MR_all.loc[MR_all['stem_len/10'] == group]['portion_STR_seqL'].dropna())
    MR_vio[str(group*10) + '-' + str(((group+1)*10)-1)] = current(vio_range)

current = stats.gaussian_kde(MR_all.loc[(MR_all['stem_len/10'] >= 10) & (MR_all['stem_len/10'] < 15)]['portion_STR_seqL'].dropna())
MR_vio['100-149'] = current(vio_range)
current = stats.gaussian_kde(MR_all.loc[(MR_all['stem_len/10'] >= 15) & (MR_all['stem_len/10'] < 20)]['portion_STR_seqL'].dropna())
MR_vio['150-199'] = current(vio_range)
current = stats.gaussian_kde(MR_all.loc[(MR_all['stem_len/10'] >= 20) & (MR_all['stem_len/10'] < 25)]['portion_STR_seqL'].dropna())
MR_vio['200-249'] = current(vio_range)
current = stats.gaussian_kde(MR_all.loc[(MR_all['stem_len/10'] >= 25) & (MR_all['stem_len/10'] < 30)]['portion_STR_seqL'].dropna())
MR_vio['250-299'] = current(vio_range)
current = stats.gaussian_kde(MR_all.loc[(MR_all['stem_len/10'] >= 30)]['portion_STR_seqL'].dropna())
MR_vio['>=300'] = current(vio_range)

KDE_all['MR'] = MR_vio

In [ ]:
random_matched_MR_all['len/10'] = random_matched_MR_all['length']//10

random_MR_vio = dict()
for group in range(1,11):
    print('\r' + str(group), end = '    ')
    current = stats.gaussian_kde(random_matched_MR_all.loc[random_matched_MR_all['len/10'] == group]['portion_STR_seq'].dropna())
    random_MR_vio[str(group*10) + '-' + str(((group+1)*10)-1)] = current(vio_range)

current = stats.gaussian_kde(random_matched_MR_all.loc[(random_matched_MR_all['len/10'] >= 10) & (random_matched_MR_all['len/10'] < 15)]['portion_STR_seq'].dropna())
random_MR_vio['100-149'] = current(vio_range)
current = stats.gaussian_kde(random_matched_MR_all.loc[(random_matched_MR_all['len/10'] >= 15) & (random_matched_MR_all['len/10'] < 20)]['portion_STR_seq'].dropna())
random_MR_vio['150-199'] = current(vio_range)
current = stats.gaussian_kde(random_matched_MR_all.loc[(random_matched_MR_all['len/10'] >= 20) & (random_matched_MR_all['len/10'] < 25)]['portion_STR_seq'].dropna())
random_MR_vio['200-249'] = current(vio_range)
current = stats.gaussian_kde(random_matched_MR_all.loc[(random_matched_MR_all['len/10'] >= 25) & (random_matched_MR_all['len/10'] < 30)]['portion_STR_seq'].dropna())
random_MR_vio['250-299'] = current(vio_range)
current = stats.gaussian_kde(random_matched_MR_all.loc[(random_matched_MR_all['len/10'] >= 30)]['portion_STR_seq'].dropna())
random_MR_vio['>=300'] = current(vio_range)

KDE_all['random_MR'] = random_MR_vio

In [ ]:
MR_hRhY['stem_len/10'] = MR_hRhY['stem_len']//10

MR_vio_hRhY = dict()
for group in range(1,10):
    print('\r' + str(group), end = '    ')
    current = stats.gaussian_kde(MR_hRhY.loc[MR_hRhY['stem_len/10'] == group]['portion_STR_seqL'].dropna())
    MR_vio_hRhY[str(group*10) + '-' + str(((group+1)*10)-1)] = current(vio_range)

current = stats.gaussian_kde(MR_hRhY.loc[(MR_hRhY['stem_len/10'] >= 10) & (MR_hRhY['stem_len/10'] < 15)]['portion_STR_seqL'].dropna())
MR_vio_hRhY['100-149'] = current(vio_range)
current = stats.gaussian_kde(MR_hRhY.loc[(MR_hRhY['stem_len/10'] >= 15) & (MR_hRhY['stem_len/10'] < 20)]['portion_STR_seqL'].dropna())
MR_vio_hRhY['150-199'] = current(vio_range)
current = stats.gaussian_kde(MR_hRhY.loc[(MR_hRhY['stem_len/10'] >= 20) & (MR_hRhY['stem_len/10'] < 25)]['portion_STR_seqL'].dropna())
MR_vio_hRhY['200-249'] = current(vio_range)
current = stats.gaussian_kde(MR_hRhY.loc[(MR_hRhY['stem_len/10'] >= 25) & (MR_hRhY['stem_len/10'] < 30)]['portion_STR_seqL'].dropna())
MR_vio_hRhY['250-299'] = current(vio_range)
current = stats.gaussian_kde(MR_hRhY.loc[(MR_hRhY['stem_len/10'] >= 30)]['portion_STR_seqL'].dropna())
MR_vio_hRhY['>=300'] = current(vio_range)

KDE_all['MR_hRhY'] = MR_vio_hRhY

In [ ]:
random_matched_MR_hRhY['len/10'] = random_matched_MR_hRhY['length']//10

random_MR_vio_hRhY = dict()
for group in range(1,11):
    print('\r' + str(group), end = '    ')
    current = stats.gaussian_kde(random_matched_MR_hRhY.loc[random_matched_MR_hRhY['len/10'] == group]['portion_STR_seq'].dropna())
    random_MR_vio_hRhY[str(group*10) + '-' + str(((group+1)*10)-1)] = current(vio_range)

current = stats.gaussian_kde(random_matched_MR_hRhY.loc[(random_matched_MR_hRhY['len/10'] >= 10) & (random_matched_MR_hRhY['len/10'] < 15)]['portion_STR_seq'].dropna())
random_MR_vio_hRhY['100-149'] = current(vio_range)
current = stats.gaussian_kde(random_matched_MR_hRhY.loc[(random_matched_MR_hRhY['len/10'] >= 15) & (random_matched_MR_hRhY['len/10'] < 20)]['portion_STR_seq'].dropna())
random_MR_vio_hRhY['150-199'] = current(vio_range)
current = stats.gaussian_kde(random_matched_MR_hRhY.loc[(random_matched_MR_hRhY['len/10'] >= 20) & (random_matched_MR_hRhY['len/10'] < 25)]['portion_STR_seq'].dropna())
random_MR_vio_hRhY['200-249'] = current(vio_range)
current = stats.gaussian_kde(random_matched_MR_hRhY.loc[(random_matched_MR_hRhY['len/10'] >= 25) & (random_matched_MR_hRhY['len/10'] < 30)]['portion_STR_seq'].dropna())
random_MR_vio_hRhY['250-299'] = current(vio_range)
current = stats.gaussian_kde(random_matched_MR_hRhY.loc[(random_matched_MR_hRhY['len/10'] >= 30)]['portion_STR_seq'].dropna())
random_MR_vio_hRhY['>=300'] = current(vio_range)

KDE_all['random_MR_hRhY'] = random_MR_vio_hRhY

In [ ]:
MR_other['stem_len/10'] = MR_other['stem_len']//10

MR_vio_other = dict()
for group in range(1,10):
    print('\r' + str(group), end = '    ')
    current = stats.gaussian_kde(MR_other.loc[MR_other['stem_len/10'] == group]['portion_STR_seqL'].dropna())
    MR_vio_other[str(group*10) + '-' + str(((group+1)*10)-1)] = current(vio_range)

current = stats.gaussian_kde(MR_other.loc[(MR_other['stem_len/10'] >= 10) & (MR_other['stem_len/10'] < 15)]['portion_STR_seqL'].dropna())
MR_vio_other['100-149'] = current(vio_range)
current = stats.gaussian_kde(MR_other.loc[(MR_other['stem_len/10'] >= 15) & (MR_other['stem_len/10'] < 20)]['portion_STR_seqL'].dropna())
MR_vio_other['150-199'] = current(vio_range)
current = stats.gaussian_kde(MR_other.loc[(MR_other['stem_len/10'] >= 20)]['portion_STR_seqL'].dropna())
MR_vio_other['>=200'] = current(vio_range)

KDE_all['MR_other'] = MR_vio_other

In [ ]:
random_matched_MR_other['len/10'] = random_matched_MR_other['length']//10

random_MR_vio_other = dict()
for group in range(1,11):
    print('\r' + str(group), end = '    ')
    current = stats.gaussian_kde(random_matched_MR_other.loc[random_matched_MR_other['len/10'] == group]['portion_STR_seq'].dropna())
    random_MR_vio_other[str(group*10) + '-' + str(((group+1)*10)-1)] = current(vio_range)

current = stats.gaussian_kde(random_matched_MR_other.loc[(random_matched_MR_other['len/10'] >= 10) & (random_matched_MR_other['len/10'] < 15)]['portion_STR_seq'].dropna())
random_MR_vio_other['100-149'] = current(vio_range)
current = stats.gaussian_kde(random_matched_MR_other.loc[(random_matched_MR_other['len/10'] >= 15) & (random_matched_MR_other['len/10'] < 20)]['portion_STR_seq'].dropna())
random_MR_vio_other['150-199'] = current(vio_range)
current = stats.gaussian_kde(random_matched_MR_other.loc[(random_matched_MR_other['len/10'] >= 20)]['portion_STR_seq'].dropna())
random_MR_vio_other['>=200'] = current(vio_range)

KDE_all['random_MR_other'] = random_MR_vio_other

# Inverted repeats

### Load IR database

In [ ]:
IR_all = pd.read_csv('repeat_db/IR.csv.gzip', low_memory=False, compression = 'gzip')

IR_all['chrom'] = [str(chrom) if chrom in ['X', 'Y'] else int(chrom) for chrom in IR_all['chrom']]
IR_all['stem_len'] = IR_all['L_end'] - IR_all['L_start']
IR_all['spacer'] = IR_all['R_start'] - IR_all['L_end']
IR_all['seq_L'] = [CHM13_genome[chrom][start: end+1] for chrom, start, end in zip(IR_all['chrom'], IR_all['L_start'], IR_all['L_end'])]

In [ ]:
# For fully-overlapping entries, keep the one with the longest stem length
# Note: step has already been done for database in Supplemental Files
IR_all = IR_all.sort_values(by = ['chrom', 'L_start', 'stem_len']).drop_duplicates(subset = ['chrom', 'L_start'], keep = 'last')

## Find STRs within IRs

In [ ]:
for n in range(1,10):
    print('\r' + str(n), end = '  ')
    IR_all['STR_'+str(n)] = [list(set(str_re[n].findall(seq))) for seq in IR_all['seq_L']]
IR_all['STR_1-9_seqL'] = IR_all['STR_1'] + IR_all['STR_2'] + IR_all['STR_3'] + IR_all['STR_4'] + IR_all['STR_5'] + IR_all['STR_6'] + IR_all['STR_7'] + IR_all['STR_8'] + IR_all['STR_9']
IR_all['STR_1-9_seqL'] = IR_all['STR_1-9_seqL'].apply(lambda x: list(set(x)))
del IR_all['STR_1']; del IR_all['STR_2']; del IR_all['STR_3']; del IR_all['STR_4']; del IR_all['STR_5']; del IR_all['STR_6']; del IR_all['STR_7']; del IR_all['STR_8']; del IR_all['STR_9']

IR_all['total_nt_STRs_max_seqL'] = [max([len(re.findall(STR, seq)) * len(STR) for STR in STRs], default = 0) for seq, STRs in zip(IR_all['seq_L'], IR_all['STR_1-9_seqL'])]
IR_all['portion_STR_seqL'] = IR_all['total_nt_STRs_max_seqL'] / IR_all['stem_len']
IR_all['diff_STR_seqL'] = IR_all['stem_len'] - IR_all['total_nt_STRs_max_seqL']

IR_portion = pd.concat([IR_all.groupby(['stem_len'])['portion_STR_seqL'].mean().sort_index(), IR_all.groupby(['stem_len'])['portion_STR_seqL'].quantile(0.975).sort_index(), IR_all.groupby(['stem_len'])['portion_STR_seqL'].quantile(0.025).sort_index()], axis=1)
IR_portion.columns = ['mean', 'high', 'low']
portion_STR['IR_all'] = IR_portion

### Random matched coordinates

In [ ]:
random_matched_IR_all = generate_random_matched_by_length(IR_all[['chrom', 'L_start', 'L_end']], start_col = 'L_start', end_col = 'L_end', length_col = 'stem_len')

In [ ]:
for n in range(1,10):
    print('\r' + str(n), end = '  ')
    random_matched_IR_all['STR_'+str(n)] = [list(set(str_re[n].findall(seq))) for seq in random_matched_IR_all['Sequence']]
random_matched_IR_all['STR_1-9_seq'] = random_matched_IR_all['STR_1'] + random_matched_IR_all['STR_2'] + random_matched_IR_all['STR_3'] + random_matched_IR_all['STR_4'] + random_matched_IR_all['STR_5'] + random_matched_IR_all['STR_6'] + random_matched_IR_all['STR_7'] + random_matched_IR_all['STR_8'] + random_matched_IR_all['STR_9']
random_matched_IR_all['STR_1-9_seq'] = random_matched_IR_all['STR_1-9_seq'].apply(lambda x: list(set(x)))
del random_matched_IR_all['STR_1']; del random_matched_IR_all['STR_2']; del random_matched_IR_all['STR_3']; del random_matched_IR_all['STR_4']; del random_matched_IR_all['STR_5']; del random_matched_IR_all['STR_6']; del random_matched_IR_all['STR_7']; del random_matched_IR_all['STR_8']; del random_matched_IR_all['STR_9']

random_matched_IR_all['total_nt_STRs_max_seq'] = [max([len(re.findall(STR, seq)) * len(STR) for STR in STRs], default = 0) for seq, STRs in zip(random_matched_IR_all['Sequence'], random_matched_IR_all['STR_1-9_seq'])]

random_matched_IR_all['portion_STR_seq'] = random_matched_IR_all['total_nt_STRs_max_seq'] / random_matched_IR_all['stem_len']
random_matched_IR_all['diff_STR_seq'] = random_matched_IR_all['stem_len'] - random_matched_IR_all['total_nt_STRs_max_seq']

random_matched_IR_portion = pd.concat([random_matched_IR_all.groupby(['stem_len'])['portion_STR_seq'].mean().sort_index(), random_matched_IR_all.groupby(['stem_len'])['portion_STR_seq'].quantile(0.975).sort_index(), random_matched_IR_all.groupby(['stem_len'])['portion_STR_seq'].quantile(0.025).sort_index()], axis=1)
random_matched_IR_portion.columns = ['mean', 'high', 'low']
portion_STR['random_IR_all'] = random_matched_IR_portion

### STR probability density

In [ ]:
IR_all['stem_len/10'] = IR_all['stem_len']//10

IR_vio = dict()
for group in range(1,10):
    print('\r' + str(group), end = '    ')
    current = stats.gaussian_kde(IR_all.loc[IR_all['stem_len/10'] == group]['portion_STR_seqL'].dropna())
    IR_vio[str(group*10) + '-' + str(((group+1)*10)-1)] = current(vio_range)

current = stats.gaussian_kde(IR_all.loc[(IR_all['stem_len/10'] >= 10) & (IR_all['stem_len/10'] < 15)]['portion_STR_seqL'].dropna())
IR_vio['100-149'] = current(vio_range)
current = stats.gaussian_kde(IR_all.loc[(IR_all['stem_len/10'] >= 15)]['portion_STR_seqL'].dropna())
IR_vio['>=150'] = current(vio_range)

KDE_all['IR'] = IR_vio

In [ ]:
random_matched_IR_all['len/10'] = random_matched_IR_all['stem_len']//10

random_IR_vio = dict()
for group in range(1,10):
    print('\r' + str(group), end = '    ')
    current = stats.gaussian_kde(random_matched_IR_all.loc[random_matched_IR_all['len/10'] == group]['portion_STR_seq'].dropna())
    random_IR_vio[str(group*10) + '-' + str(((group+1)*10)-1)] = current(vio_range)

current = stats.gaussian_kde(random_matched_IR_all.loc[(random_matched_IR_all['len/10'] >= 10) & (random_matched_IR_all['len/10'] < 15)]['portion_STR_seq'].dropna())
random_IR_vio['100-149'] = current(vio_range)
current = stats.gaussian_kde(random_matched_IR_all.loc[(random_matched_IR_all['len/10'] >= 15)]['portion_STR_seq'].dropna())
random_IR_vio['>=150'] = current(vio_range)

KDE_all['random_IR'] = random_IR_vio

# Direct repeats

### Load DR database

In [ ]:
DR_all = pd.read_csv('repeat_db/IR.csv.gzip', low_memory=False, compression = 'gzip')

DR_all['chrom'] = [str(chrom) if chrom in ['X', 'Y'] else int(chrom) for chrom in DR_all['chrom']]
DR_all['stem_len'] = DR_all['L_end'] - DR_all['L_start']
DR_all['spacer'] = DR_all['R_start'] - DR_all['L_end']
DR_all['seq_L'] = [CHM13_genome[chrom][start: end+1] for chrom, start, end in zip(DR_all['chrom'], DR_all['L_start'], DR_all['L_end'])]

In [ ]:
# Remove entries with spacer length >100
# For fully-overlapping entries, keep the one with the longest stem length
# Note: step has already been done for database in Supplemental Files
DR_all = DR_all.loc[DR_all['spacer'] <=100].sort_values(by = ['chrom', 'L_start', 'stem_len']).drop_duplicates(subset = ['chrom', 'L_start'], keep = 'last')

## Find STRs within DRs

In [ ]:
for n in range(1,10):
    print('\r' + str(n), end = '  ')
    DR_all['STR_'+str(n)] = [list(set(str_re[n].findall(seq))) for seq in DR_all['seq_L']]
DR_all['STR_1-9_seqL'] = DR_all['STR_1'] + DR_all['STR_2'] + DR_all['STR_3'] + DR_all['STR_4'] + DR_all['STR_5'] + DR_all['STR_6'] + DR_all['STR_7'] + DR_all['STR_8'] + DR_all['STR_9']
DR_all['STR_1-9_seqL'] = DR_all['STR_1-9_seqL'].apply(lambda x: list(set(x)))
del DR_all['STR_1']; del DR_all['STR_2']; del DR_all['STR_3']; del DR_all['STR_4']; del DR_all['STR_5']; del DR_all['STR_6']; del DR_all['STR_7']; del DR_all['STR_8']; del DR_all['STR_9']
DR_all['total_nt_STRs_max_seqL'] = [max([len(re.findall(STR, seq)) * len(STR) for STR in STRs], default = 0) for seq, STRs in zip(DR_all['seq_L'], DR_all['STR_1-9_seqL'])]
DR_all['portion_STR_seqL'] = DR_all['total_nt_STRs_max_seqL'] / DR_all['stem_len']
DR_all['diff_STR_seqL'] = DR_all['stem_len'] - DR_all['total_nt_STRs_max_seqL']

DR_portion = pd.concat([DR_all.groupby(['stem_len'])['portion_STR_seqL'].mean(), DR_all.groupby(['stem_len'])['portion_STR_seqL'].quantile(0.975), DR_all.groupby(['stem_len'])['portion_STR_seqL'].quantile(0.025)], axis=1)
DR_portion.columns = ['mean', 'high', 'low']
portion_STR['DR_all'] = DR_portion

### Random matched coordinates

In [ ]:
random_matched_DR_all = generate_random_matched_by_length(DR_all[['chrom', 'L_start', 'L_end']], start_col = 'L_start', end_col = 'L_end')

In [ ]:
# find all repeating elements of unit length 1-9 in each MR stem
for n in range(1,10):
    print('\r' + str(n), end = '  ')
    random_matched_DR_all['STR_'+str(n)] = [list(set(str_re[n].findall(seq))) for seq in random_matched_DR_all['Sequence']]
random_matched_DR_all['STR_1-9'] = random_matched_DR_all['STR_1'] + random_matched_DR_all['STR_2'] + random_matched_DR_all['STR_3'] + random_matched_DR_all['STR_4'] + random_matched_DR_all['STR_5'] + random_matched_DR_all['STR_6'] + random_matched_DR_all['STR_7'] + random_matched_DR_all['STR_8'] + random_matched_DR_all['STR_9']
random_matched_DR_all['STR_1-9'] = random_matched_DR_all['STR_1-9'].apply(lambda x: list(set(x)))
del random_matched_DR_all['STR_1']; del random_matched_DR_all['STR_2']; del random_matched_DR_all['STR_3']; del random_matched_DR_all['STR_4']; del random_matched_DR_all['STR_5']; del random_matched_DR_all['STR_6']; del random_matched_DR_all['STR_7']; del random_matched_DR_all['STR_8']; del random_matched_DR_all['STR_9']
random_matched_DR_all['total_nt_STRs'] = [[len(re.findall(STR, seq)) * len(STR) for STR in STRs] for seq, STRs in zip(random_matched_DR_all['Sequence'], random_matched_DR_all['STR_1-9'])]
random_matched_DR_all['total_nt_STRs_max'] = random_matched_DR_all['total_nt_STRs'].apply(lambda x: pd.Series(x).max())
random_matched_DR_all['portion_STR'] = random_matched_DR_all['total_nt_STRs_max'] / random_matched_DR_all['length']

random_matched_DR_portion = pd.concat([random_matched_DR_all.groupby(['length'])['portion_STR'].mean(), random_matched_DR_all.groupby(['length'])['portion_STR'].quantile(0.975), random_matched_DR_all.groupby(['length'])['portion_STR'].quantile(0.025)], axis=1)
random_matched_DR_portion.columns = ['mean', 'high', 'low']
portion_STR['random_DR_all'] = random_matched_DR_portion

### STR probability density

In [ ]:
DR_all['stem_len/10'] = DR_all['stem_len']//10

DR_vio = dict()
for group in range(1,21):
    print('\r' + str(group), end = '    ')
    current = stats.gaussian_kde(DR_all.loc[DR_all['stem_len/10'] == group]['portion_STR_seqL'].dropna())
    DR_vio[group] = current(vio_range)

DR_vio = dict()
for group in range(1,10):
    print('\r' + str(group), end = '    ')
    current = stats.gaussian_kde(DR_all.loc[DR_all['stem_len/10'] == group]['portion_STR_seqL'].dropna())
    DR_vio[str(group*10) + '-' + str(((group+1)*10)-1)] = current(vio_range)

current = stats.gaussian_kde(DR_all.loc[(DR_all['stem_len/10'] >= 10) & (DR_all['stem_len/10'] < 20)]['portion_STR_seqL'].dropna())
DR_vio['100-199'] = current(vio_range)
current = stats.gaussian_kde(DR_all.loc[(DR_all['stem_len/10'] >= 20) & (DR_all['stem_len/10'] < 30)]['portion_STR_seqL'].dropna())
DR_vio['200-299'] = current(vio_range)
current = stats.gaussian_kde(DR_all.loc[(DR_all['stem_len/10'] >= 30) & (DR_all['stem_len/10'] < 40)]['portion_STR_seqL'].dropna())
DR_vio['300-399'] = current(vio_range)
current = stats.gaussian_kde(DR_all.loc[(DR_all['stem_len/10'] >= 40) & (DR_all['stem_len/10'] < 50)]['portion_STR_seqL'].dropna())
DR_vio['400-499'] = current(vio_range)
current = stats.gaussian_kde(DR_all.loc[(DR_all['stem_len/10'] >= 50)]['portion_STR_seqL'].dropna())
DR_vio['>=500'] = current(vio_range)

KDE_all['DR'] = DR_vio

In [ ]:
random_matched_DR_all['len/10'] = random_matched_DR_all['length']//10

random_DR_vio = dict()
for group in range(1,10):
    print('\r' + str(group), end = '    ')
    current = stats.gaussian_kde(random_matched_DR_all.loc[random_matched_DR_all['len/10'] == group]['portion_STR'].dropna())
    random_DR_vio[str(group*10) + '-' + str(((group+1)*10)-1)] = current(vio_range)

current = stats.gaussian_kde(random_matched_DR_all.loc[(random_matched_DR_all['len/10'] >= 10) & (random_matched_DR_all['len/10'] < 20)]['portion_STR'].dropna())
random_DR_vio['100-199'] = current(vio_range)
current = stats.gaussian_kde(random_matched_DR_all.loc[(random_matched_DR_all['len/10'] >= 20) & (random_matched_DR_all['len/10'] < 30)]['portion_STR'].dropna())
random_DR_vio['200-299'] = current(vio_range)
current = stats.gaussian_kde(random_matched_DR_all.loc[(random_matched_DR_all['len/10'] >= 30) & (random_matched_DR_all['len/10'] < 40)]['portion_STR'].dropna())
random_DR_vio['300-399'] = current(vio_range)
current = stats.gaussian_kde(random_matched_DR_all.loc[(random_matched_DR_all['len/10'] >= 40) & (random_matched_DR_all['len/10'] < 50)]['portion_STR'].dropna())
random_DR_vio['400-499'] = current(vio_range)
current = stats.gaussian_kde(random_matched_DR_all.loc[(random_matched_DR_all['len/10'] >= 50)]['portion_STR'].dropna())
random_DR_vio['>=500'] = current(vio_range)

KDE_all['random_DR'] = random_DR_vio

# G4 DNA (GRCh38)

### Load G4 database

In [ ]:
G4seq_all = pd.read_csv('repeat_db/G4_GRCh38.csv.gzip', compression = 'gzip')

In [ ]:
# For fully-overlapping entries, keep the one with the longest stem length
# Note: step has already been done for database in Supplemental Files
G4seq_all = G4seq_all.sort_values(by = ['chrom', 'start', 'length']).drop_duplicates(subset = ['chrom', 'start'], keep = 'last')

## Find STRs within G4s

In [ ]:
# find all repeating elements of unit length 1-9 in each MR stem
for n in range(1,10):
    print('\r' + str(n), end = '  ')
    G4seq_all['STR_'+str(n)] = [list(set(str_re[n].findall(seq))) for seq in G4seq_all['Sequence']]
G4seq_all['STR_1-9_seq'] = G4seq_all['STR_1'] + G4seq_all['STR_2'] + G4seq_all['STR_3'] + G4seq_all['STR_4'] + G4seq_all['STR_5'] + G4seq_all['STR_6'] + G4seq_all['STR_7'] + G4seq_all['STR_8'] + G4seq_all['STR_9']
G4seq_all['STR_1-9_seq'] = G4seq_all['STR_1-9_seq'].apply(lambda x: list(set(x)))
del G4seq_all['STR_1']; del G4seq_all['STR_2']; del G4seq_all['STR_3']; del G4seq_all['STR_4']; del G4seq_all['STR_5']; del G4seq_all['STR_6']; del G4seq_all['STR_7']; del G4seq_all['STR_8']; del G4seq_all['STR_9']
G4seq_all['total_nt_STRs_max_seq'] = [max([len(re.findall(STR, seq)) * len(STR) for STR in STRs], default = 0) for seq, STRs in zip(G4seq_all['Sequence'], G4seq_all['STR_1-9_seq'])]
G4seq_all['portion_STR_seq'] = G4seq_all['total_nt_STRs_max_seq'] / G4seq_all['length']
G4seq_all['diff_STR_seq'] = G4seq_all['length'] - G4seq_all['total_nt_STRs_max_seq']

G4seq_portion = pd.concat([G4seq_all.groupby(['length'])['portion_STR_seq'].mean(), G4seq_all.groupby(['length'])['portion_STR_seq'].quantile(0.975), G4seq_all.groupby(['length'])['portion_STR_seq'].quantile(0.025)], axis=1)
G4seq_portion.columns = ['mean', 'high', 'low']
portion_STR['G4'] = G4seq_portion

### Random matched coordinates

In [ ]:
random_matched_G4seq_all = generate_random_matched_by_length(G4seq_all[['chrom', 'start', 'end']], start_col = 'start', end_col = 'end')

In [ ]:
# find all repeating elements of unit length 1-9 in each MR stem
for n in range(1,10):
    print('\r' + str(n), end = '  ')
    random_matched_G4seq_all['STR_'+str(n)] = [list(set(str_re[n].findall(seq))) for seq in random_matched_G4seq_all['Sequence']]
random_matched_G4seq_all['STR_1-9'] = random_matched_G4seq_all['STR_1'] + random_matched_G4seq_all['STR_2'] + random_matched_G4seq_all['STR_3'] + random_matched_G4seq_all['STR_4'] + random_matched_G4seq_all['STR_5'] + random_matched_G4seq_all['STR_6'] + random_matched_G4seq_all['STR_7'] + random_matched_G4seq_all['STR_8'] + random_matched_G4seq_all['STR_9']
random_matched_G4seq_all['STR_1-9'] = random_matched_G4seq_all['STR_1-9'].apply(lambda x: list(set(x)))
del random_matched_G4seq_all['STR_1']; del random_matched_G4seq_all['STR_2']; del random_matched_G4seq_all['STR_3']; del random_matched_G4seq_all['STR_4']; del random_matched_G4seq_all['STR_5']; del random_matched_G4seq_all['STR_6']; del random_matched_G4seq_all['STR_7']; del random_matched_G4seq_all['STR_8']; del random_matched_G4seq_all['STR_9']
random_matched_G4seq_all['total_nt_STRs_max_seq'] = [max([len(re.findall(STR, seq)) * len(STR) for STR in STRs], default = 0) for seq, STRs in zip(random_matched_G4seq_all['Sequence'], random_matched_G4seq_all['STR_1-9'])]
random_matched_G4seq_all['portion_STR'] = random_matched_G4seq_all['total_nt_STRs_max_seq'] / random_matched_G4seq_all['length']

random_matched_G4seq_portion = pd.concat([random_matched_G4seq_all.groupby(['length'])['portion_STR'].mean(), random_matched_G4seq_all.groupby(['length'])['portion_STR'].quantile(0.975), random_matched_G4seq_all.groupby(['length'])['portion_STR'].quantile(0.025)], axis=1)
random_matched_G4seq_portion.columns = ['mean', 'high', 'low']
portion_STR['random_G4'] = random_matched_G4seq_portion

### STR probability density

In [ ]:
G4seq_all['len/10'] = G4seq_all['length']//10

G4_vio = dict()
for group in range(1,10):
    print('\r' + str(group), end = '    ')
    current = stats.gaussian_kde(G4seq_all.loc[G4seq_all['len/10'] == group]['portion_STR_seq'].dropna())
    G4_vio[str(group*10) + '-' + str(((group+1)*10)-1)] = current(vio_range)

current = stats.gaussian_kde(G4seq_all.loc[(G4seq_all['len/10'] >= 10) & (G4seq_all['len/10'] < 20)]['portion_STR_seq'].dropna())
G4_vio['100-199'] = current(vio_range)
current = stats.gaussian_kde(G4seq_all.loc[(G4seq_all['len/10'] >= 20) & (G4seq_all['len/10'] < 30)]['portion_STR_seq'].dropna())
G4_vio['200-299'] = current(vio_range)
current = stats.gaussian_kde(G4seq_all.loc[(G4seq_all['len/10'] >= 30) & (G4seq_all['len/10'] < 40)]['portion_STR_seq'].dropna())
G4_vio['300-399'] = current(vio_range)
current = stats.gaussian_kde(G4seq_all.loc[(G4seq_all['len/10'] >= 40) & (G4seq_all['len/10'] < 50)]['portion_STR_seq'].dropna())
G4_vio['400-499'] = current(vio_range)
current = stats.gaussian_kde(G4seq_all.loc[(G4seq_all['len/10'] >= 50)]['portion_STR_seq'].dropna())
G4_vio['>=500'] = current(vio_range)

KDE_all['G4'] = G4_vio

In [ ]:
random_matched_G4seq_all['len/10'] = random_matched_G4seq_all['length']//10

random_matched_G4_vio = dict()
for group in range(1,10):
    print('\r' + str(group), end = '    ')
    current = stats.gaussian_kde(random_matched_G4seq_all.loc[random_matched_G4seq_all['len/10'] == group]['portion_STR'].dropna())
    random_matched_G4_vio[str(group*10) + '-' + str(((group+1)*10)-1)] = current(vio_range)

current = stats.gaussian_kde(random_matched_G4seq_all.loc[(random_matched_G4seq_all['len/10'] >= 10) & (random_matched_G4seq_all['len/10'] < 20)]['portion_STR'].dropna())
random_matched_G4_vio['100-199'] = current(vio_range)
current = stats.gaussian_kde(random_matched_G4seq_all.loc[(random_matched_G4seq_all['len/10'] >= 20) & (random_matched_G4seq_all['len/10'] < 30)]['portion_STR'].dropna())
random_matched_G4_vio['200-299'] = current(vio_range)
current = stats.gaussian_kde(random_matched_G4seq_all.loc[(random_matched_G4seq_all['len/10'] >= 30) & (random_matched_G4seq_all['len/10'] < 40)]['portion_STR'].dropna())
random_matched_G4_vio['300-399'] = current(vio_range)
current = stats.gaussian_kde(random_matched_G4seq_all.loc[(random_matched_G4seq_all['len/10'] >= 40) & (random_matched_G4seq_all['len/10'] < 50)]['portion_STR'].dropna())
random_matched_G4_vio['400-499'] = current(vio_range)
current = stats.gaussian_kde(random_matched_G4seq_all.loc[(random_matched_G4seq_all['len/10'] >= 50)]['portion_STR'].dropna())
random_matched_G4_vio['>=500'] = current(vio_range)

KDE_all['random_G4'] = random_matched_G4_vio

# Save/load portion data

In [ ]:
portion_STR.keys()

In [ ]:
# Save
with open('../../paper_code_container/custom_db/portion_STRs_pub.pickle', 'wb') as handle:
    pickle.dump(portion_STR, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Save
with open('../../paper_code_container/custom_db/KDE_STRs_pub.pickle', 'wb') as handle:
    pickle.dump(KDE_all, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Load
with open('../../paper_code_container/custom_db/portion_STRs_pub.pickle', 'rb') as handle:
    portion_STR = pickle.load(handle)

In [ ]:
# Load
with open('../../paper_code_container/custom_db/KDE_STRs_pub.pickle', 'rb') as handle:
    KDE_all = pickle.load(handle)

# Plots of length vs. STR portion

In [ ]:
def make_plot_length_portion(cat_name, random_name, legend_text, show = True, legendpos = 0.75, xrange = [10, 405]):
    fig_STR = go.Figure()
    fig_STR.add_trace(go.Scatter(x = portion_STR[cat_name].index, y = portion_STR[cat_name]['mean'], line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[0], width = 3), name = legend_text, legendgroup = legend_text))
    fig_STR.add_trace(go.Scatter(x = portion_STR[cat_name].index, y = portion_STR[cat_name]['high'], line = dict(color = 'rgba(0,0,0,0)'), legendgroup = legend_text, showlegend = False, name = 'quantile 95%'))
    fig_STR.add_trace(go.Scatter(x = portion_STR[cat_name].index, y = portion_STR[cat_name]['low'], fill='tonexty', fillcolor = 'rgba(31, 119, 180,0.25)', line = dict(color = 'rgba(0,0,0,0)'), legendgroup = legend_text, showlegend = False, name = 'quantile 5%'))
    
    fig_STR.add_trace(go.Scatter(x = portion_STR[random_name].index, y = portion_STR[random_name]['mean'], line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[1], width = 3),  connectgaps = True, name = 'random coordinates', legendgroup = 'random coordinates'))
    fig_STR.add_trace(go.Scatter(x = portion_STR[random_name].index, y = portion_STR[random_name]['high'], line = dict(color = 'rgba(0,0,0,0)'), connectgaps = True, legendgroup = 'random coordinates', showlegend = False, name = 'quantile 95%'))
    fig_STR.add_trace(go.Scatter(x = portion_STR[random_name].index, y = portion_STR[random_name]['low'], fill='tonexty', fillcolor = 'rgba(255,127,14,0.25)', line = dict(color = 'rgba(0,0,0,0)'), connectgaps = True, legendgroup = 'random coordinates', showlegend = False, name = 'quantile 5%'))
    
    fig_STR.update_xaxes(title = 'mirror repeat stem length (nt)', range = xrange)
    fig_STR.update_yaxes(range = [0.2, 1.05], title = 'STR portion of stem length')
    fig_STR.update_layout(font=dict(family = 'Arial', size = 14), legend = dict(yanchor="top", y=legendpos, xanchor="right", x=0.99), height = 300, width = 600, margin={'t':20,'l':55,'b':35,'r':10})
    if show == True:
        fig_STR.show()
    return fig_STR

### MRs

In [ ]:
fig_STR_MR = make_plot_length_portion('MR', 'random_MR', 'mirror repeats')

In [ ]:
fig_STR_MR.write_image('../../papers/this lab/mirror_repeats/plots/MR_portion_bylength.pdf')

In [ ]:
fig_STR_MR_hRhY = make_plot_length_portion('MR_hRhY', 'random_MR_hRhY', 'hR/hY mirror repeats')

In [ ]:
fig_STR_MR_hRhY.write_image('../../papers/this lab/mirror_repeats/plots/MR_portion_bylength_hRhY.pdf')

In [ ]:
fig_STR_MR_other = make_plot_length_portion('MR_other', 'random_MR_other', 'other mirror repeats', legendpos= 0.65, xrange = [10,305])

In [ ]:
fig_STR_MR_other.write_image('../../papers/this lab/mirror_repeats/plots/MR_portion_bylength_other.pdf')

### IRs

In [ ]:
fig_STR_IR = make_plot_length_portion('IR_all', 'random_IR_all', 'inverted repeats', legendpos= 0.79, xrange = [10,335])

In [ ]:
fig_STR_IR.write_image('../../papers/this lab/mirror_repeats/plots/IR_portion_bylength.pdf')

### DRs

In [ ]:
fig_STR_DR = make_plot_length_portion('DR_all', 'random_DR_all', 'direct repeats', legendpos= 0.95, xrange = [10,975])

In [ ]:
fig_STR_DR.write_image('../../papers/this lab/mirror_repeats/plots/DR_portion_bylength.pdf')

### G4s

In [ ]:
fig_STR_G4 = make_plot_length_portion('G4', 'random_G4', 'G4 motifs', legendpos= 0.95, xrange = [10,1000])

In [ ]:
fig_STR_G4.write_image('../../papers/this lab/mirror_repeats/plots/G4_portion_bylength.pdf')

# Plots of STR probability density

In [ ]:
def make_prob_density_plot(cat_name, random_name, title_text, show = True):
    fig = make_subplots(rows = 1, cols = 2, column_titles = [title_text, 'random coordinates'], x_title = 'probability density', shared_yaxes = True)
    colorscale = plotly.colors.sample_colorscale('Turbo', len(KDE_all[cat_name]))
    colornum = 0
    for group in KDE_all[cat_name]:
        fig.add_trace(go.Scatter(y = vio_range, x = KDE_all[cat_name][group] / KDE_all[cat_name][group].sum(), line = dict(color = colorscale[colornum]), opacity = 0.3 + (0.03*colornum), legendgroup = group, name = group), row = 1, col = 1)
        fig.add_trace(go.Scatter(y = vio_range, x = KDE_all[random_name][group] / KDE_all[random_name][group].sum(), line = dict(color = colorscale[colornum]), opacity = 0.3 + (0.03*colornum), legendgroup = group, showlegend = False, name = group), row = 1, col = 2)
        colornum +=1
    fig.update_xaxes(gridcolor = 'rgba(0,0,0,0.15)', gridwidth = 1)
    fig.update_yaxes(range = [0.1675, 1.05], gridcolor = 'rgba(0,0,0,0.15)', gridwidth = 1)
    fig.update_yaxes(title = 'STR portion of motif length', row = 1, col = 1)
    fig.update_layout(legend_title = 'length (nt)', font=dict(family = 'Arial', size = 14), height = 600, width = 800, margin={'t':40,'l':50,'b':50,'r':10})     
    if show == True:
        fig.show()
    return fig

### MRs

In [ ]:
fig_prob_MR = make_prob_density_plot('MR', 'random_MR', 'mirror repeats')

In [ ]:
fig_prob_MR.write_image('../../papers/this lab/mirror_repeats/plots/KDE_MR.pdf')

In [ ]:
fig_prob_MR_hRhY = make_prob_density_plot('MR_hRhY', 'random_MR_hRhY', 'hR/hY mirror repeats')

In [ ]:
fig_prob_MR_hRhY.write_image('../../papers/this lab/mirror_repeats/plots/KDE_MR_hRhY.pdf')

In [ ]:
fig_prob_MR_hRhY = make_prob_density_plot('MR_other', 'random_MR_other', 'other mirror repeats')

In [ ]:
fig_prob_MR_hRhY.write_image('../../papers/this lab/mirror_repeats/plots/KDE_MR_other.pdf')

### IRs

In [ ]:
fig_prob_IR = make_prob_density_plot('IR', 'random_IR', 'inverted repeats')

In [ ]:
fig_prob_IR.write_image('../../papers/this lab/mirror_repeats/plots/KDE_IR.pdf')

### DRs

In [ ]:
fig_prob_DR = make_prob_density_plot('DR', 'random_DR', 'direct repeats')

In [ ]:
fig_prob_DR.write_image('../../papers/this lab/mirror_repeats/plots/KDE_DR.pdf')

### G4s

In [ ]:
fig_prob_G4 = make_prob_density_plot('G4', 'random_G4', 'G4 motifs')

In [ ]:
fig_prob_G4.write_image('../../papers/this lab/mirror_repeats/plots/KDE_G4.pdf')

# Motif counts

In [ ]:
MR_counts = MR_all.groupby(['stem_len', '#MM'])['chrom'].count().unstack()
IR_counts = IR_all.groupby(['stem_len', '#MM'])['chrom'].count().unstack()
DR_counts = DR_all.groupby(['stem_len', '#MM'])['chrom'].count().unstack()
G4_counts = G4seq_all.groupby(['length'])['chrom'].count()

In [ ]:
# Distributions for MRs with certain sequence composition requirements
MR_hAT = MR_all.loc[(MR_all['%AT'] > 0.95) | (MR_all['%AT'] < 0.05)].copy()
MR_hAC = MR_all.loc[(MR_all['%AC'] > 0.95) | (MR_all['%AC'] < 0.05)].copy()

MR_hRhY_counts = MR_hRhY.groupby(['stem_len', '#MM'])['chrom'].count().unstack()
MR_hAT_counts = MR_hAT.groupby(['stem_len', '#MM'])['chrom'].count().unstack()
MR_hAC_counts = MR_hAC.groupby(['stem_len', '#MM'])['chrom'].count().unstack()

MR_other_counts = MR_other.groupby(['stem_len', '#MM'])['chrom'].count().unstack()

In [ ]:
# perfect motifs only
perfect_counts = pd.concat([MR_counts[0], IR_counts[0], DR_counts[0], G4_counts, MR_hRhY_counts[0], MR_hAT_counts[0], MR_hAC_counts[0], MR_other_counts[0]], axis=1).sort_index()
perfect_counts.columns = ['mirror repeats', 'inverted repeats', 'direct repeats', 'G4 motifs', 'hR/hY mirror repeats', 'hAT mirror repeats', 'hAC mirror repeats', 'other mirror repeats']

perfect_counts_simple = pd.concat([MR_counts[0], IR_counts[0], DR_counts[0], G4_counts], axis=1).sort_index()
perfect_counts_simple.columns = ['mirror repeats', 'inverted repeats', 'direct repeats', 'G4 motifs']

## STRs

#### Load counts from previous study (McGinty et al, 2025)

In [ ]:
CHM13_counts = pd.read_pickle('repeat_distributions/CHM13_counts.pickle').sort_index()

#### Group STR motifs by unit length

In [ ]:
counts_by_unit_length = dict()
for i in range(1,5):
    counts_by_unit_length[i] = dict()
for motif in CHM13_counts['A'].columns[:-1]:
    counts_by_unit_length[len(motif)][motif] = CHM13_counts['A'][motif]
for i in range(1,5):
    counts_by_unit_length[i] = pd.concat(counts_by_unit_length[i], axis=1)
    counts_by_unit_length[i]['all'] = counts_by_unit_length[i].sum(axis=1)

#### Group STR motifs according to nucleotide content


In [ ]:
CHM13_counts_nt = dict()
for motif in CHM13_counts['A'].columns:
    CHM13_counts_nt[motif] = CHM13_counts['A'][motif]
    CHM13_counts_nt[motif].index = CHM13_counts_nt[motif].index * len(motif)
CHM13_counts_nt = pd.concat(CHM13_counts_nt, axis=1)

STR_counts_ntcontent = dict()
STR_counts_ntcontent['AG'] = CHM13_counts_nt[['AG', 'AAG', 'AGG', 'AAAG', 'AAGG', 'AGGG']].replace(0, np.nan).dropna(how = 'all').interpolate(method = 'slinear').sum(axis=1).replace(0, np.nan).dropna()
STR_counts_ntcontent['AC'] = CHM13_counts_nt[['AC', 'AAC', 'ACC', 'AAAC', 'AACC', 'ACCC']].replace(0, np.nan).dropna(how = 'all').interpolate(method = 'slinear').sum(axis=1).replace(0, np.nan).dropna()
STR_counts_ntcontent['AT'] = CHM13_counts_nt[['AT', 'AAT', 'AAAT', 'AATT']].replace(0, np.nan).dropna(how = 'all').interpolate(method = 'slinear').sum(axis=1).replace(0, np.nan).dropna()
STR_counts_ntcontent['other'] = CHM13_counts_nt[['CG', 'ATC', 'ACT', 'AGC', 'ACG', 'CCG', 'AATG', 'AATC', 'AAGT', 'AAGC', 'AACT', 'AACG', 'AGAT', 'ACAT', 'ATCC', 'ACAG', 'ACTC', 'ACTG', 'ACCT', 'AGGC', 'AGCC', 'ACGG', 'ACGC', 'ACCG', 'AGCG', 'CCCG', 'ATGC', 'ATCG', 'AGCT', 'ACGT', 'CCGG']].replace(0, np.nan).dropna(how = 'all').interpolate(method = 'slinear').sum(axis=1).replace(0, np.nan).dropna()
STR_counts_ntcontent['all'] = CHM13_counts_nt[['A', 'C', 'AC', 'AT', 'AG', 'CG', 'AAT', 'AAG', 'AAC', 'ATC', 'ACT', 'AGG', 'AGC', 'ACG', 'ACC', 'CCG', 'AAAT', 'AAAG', 'AAAC', 'AATG', 'AATC', 'AAGT', 'AAGG', 'AAGC', 'AACT', 'AACG', 'AACC', 'AGAT', 'ACAT', 'ATCC', 'ACAG', 'ACTC', 'ACTG', 'ACCT', 'AGGG', 'AGGC', 'AGCC', 'ACGG', 'ACGC', 'ACCG', 'ACCC', 'AGCG', 'CCCG', 'AATT', 'ATGC', 'ATCG', 'AGCT', 'ACGT', 'CCGG']].replace(0, np.nan).dropna(how = 'all').interpolate(method = 'slinear').sum(axis=1).replace(0, np.nan).dropna()

## MR count expectation in a randomized genome

In [ ]:
def calculate_skew(sequence, count_in = False):
    if count_in == False:
        seq_counts = pd.Series([sequence.count('A'), sequence.count('T'), sequence.count('G'), sequence.count('C')], index = ['A', 'T', 'G', 'C'])
    else:
        seq_counts = sequence
    return 1/pd.Series([(count / seq_counts.sum())**2 for count in seq_counts]).sum()

base_counts = dict()
for chrom in chrom_list_XY:
    base_counts[chrom] = pd.Series([CHM13_genome[chrom].count('A'), CHM13_genome[chrom].count('T'), CHM13_genome[chrom].count('G'), CHM13_genome[chrom].count('C')], index = ['A', 'T', 'G', 'C'])
base_counts = pd.concat(base_counts).unstack()

# Same, using actual base counts in CHM13.
base_counts_sum = pd.Series([base_counts.sum()['A'], base_counts.sum()['T'], base_counts.sum()['G'], base_counts.sum()['C']], index = ['A', 'T', 'G', 'C'])
skew_CHM13 = calculate_skew(base_counts_sum, count_in=True)

chrom_lengths = pd.Series([len(CHM13_genome[seq]) for seq in CHM13_genome], index = chrom_list_XY)
MR_expectation_CHM13 = pd.Series([(chrom_lengths.sum()) * ((1/skew_CHM13)**length) for length in range(1,17)], index = list(range(1,17)))

## Plots

In [ ]:
fig_starting_counts = go.Figure()
for i in range(1,5):
    fig_starting_counts.add_trace(go.Scatter(x = counts_by_unit_length[i].index * i, y = counts_by_unit_length[i]['all'].loc[counts_by_unit_length[i]['all'] >= 1], legendgroup = i, name = 'STR unit = '+str(i), line = dict(width = 3, color = plotly.colors.DEFAULT_PLOTLY_COLORS[i-1]), mode = 'lines', opacity = 0.5))
for col in perfect_counts_simple:
    fig_starting_counts.add_trace(go.Scatter(x = perfect_counts.index, y = perfect_counts[col].replace(0, np.nan), line = dict(width = 4 if col == 'mirror repeats' else 3), opacity = 0.9 if col == 'mirror repeats' else 0.4, connectgaps = False, name = col))
fig_starting_counts.add_trace(go.Scatter(x = MR_expectation_CHM13.index, y = MR_expectation_CHM13, line = dict(dash = 'dash', width = 3, color = plotly.colors.DEFAULT_PLOTLY_COLORS[4]), mode = 'lines', opacity = 0.75, name = 'mirror (expectation)'))

fig_starting_counts.update_xaxes(type = 'log', title = 'repeat length (nt)', range = [0,3.1])
fig_starting_counts.update_yaxes(type = 'log', title = 'counts', tickformat = '1.0e', dtick = 2)
fig_starting_counts.update_layout(font=dict(family = 'Helvetica', size = 14), legend = dict(yanchor="top", y=0.99, xanchor="right", x=1.1), height = 440, width = 800, margin={'t':20,'l':60,'b':40,'r':40})        
fig_starting_counts.show()

In [ ]:
fig_starting_counts.write_image('../../papers/this lab/mirror_repeats/plots/distributions.pdf')

In [ ]:
fig_starting_counts = go.Figure()
fig_starting_counts.add_trace(go.Scatter(x = perfect_counts.index, y = perfect_counts['hR/hY mirror repeats'].replace(0, np.nan), line = dict(width = 3), opacity = 0.9, connectgaps = False, name = 'hR/hY mirror repeats'))
fig_starting_counts.add_trace(go.Scatter(x = perfect_counts.index, y = perfect_counts['hAT mirror repeats'].replace(0, np.nan), line = dict(width = 3), opacity = 0.9, connectgaps = False, name = 'hW mirror repeats'))
fig_starting_counts.add_trace(go.Scatter(x = perfect_counts.index, y = perfect_counts['hAC mirror repeats'].replace(0, np.nan), line = dict(width = 3), opacity = 0.9, connectgaps = False, name = 'hK/hM mirror repeats'))
fig_starting_counts.add_trace(go.Scatter(x = perfect_counts.index, y = perfect_counts['other mirror repeats'].replace(0, np.nan), line = dict(width = 3), opacity = 0.9, connectgaps = False, name = 'all other mirror repeats'))

fig_starting_counts.update_xaxes(type = 'log', title = 'repeat length (nt)', range = [1,2.5])
fig_starting_counts.update_yaxes(type = 'log', title = 'counts', tickformat = '1.0e', dtick = 2)
fig_starting_counts.update_layout(font=dict(family = 'Helvetica', size = 14), legend = dict(yanchor="top", y=0.99, xanchor="right", x=1.1), height = 240, width = 800, margin={'t':20,'l':60,'b':40,'r':40})        
fig_starting_counts.show()

In [ ]:
fig_starting_counts.write_image('../../papers/this lab/mirror_repeats/plots/distributions_hRhY.pdf')

In [ ]:
fig_starting_counts = go.Figure()
fig_starting_counts.add_trace(go.Scatter(x = STR_counts_ntcontent['AG'].index, y = STR_counts_ntcontent['AG'], line = dict(width = 3), opacity = 0.9, connectgaps = False, name = 'A<sub>n</sub>G<sub>n</sub> STRs'))
fig_starting_counts.add_trace(go.Scatter(x = STR_counts_ntcontent['AT'].index, y = STR_counts_ntcontent['AT'], line = dict(width = 3), opacity = 0.9, connectgaps = False, name = 'A<sub>n</sub>T<sub>n</sub> STRs'))
fig_starting_counts.add_trace(go.Scatter(x = STR_counts_ntcontent['AC'].index, y = STR_counts_ntcontent['AC'], line = dict(width = 3), opacity = 0.9, connectgaps = False, name = 'A<sub>n</sub>C<sub>n</sub> STRs'))
fig_starting_counts.add_trace(go.Scatter(x = STR_counts_ntcontent['other'].index, y = STR_counts_ntcontent['other'], line = dict(width = 3), opacity = 0.9, connectgaps = False, name = 'other STRs'))
#fig_starting_counts.add_trace(go.Scatter(x = STR_counts_ntcontent['all'].index, y = STR_counts_ntcontent['all'], line = dict(width = 3), opacity = 0.9, connectgaps = False, name = 'all (1-4)'))


fig_starting_counts.update_xaxes(type = 'log', title = 'repeat length (nt)', range = [1,2.5])
fig_starting_counts.update_yaxes(type = 'log', range = [0, 6.5], title = 'counts', tickformat = '1.0e', dtick = 2)
fig_starting_counts.update_layout(font=dict(family = 'Helvetica', size = 14), legend = dict(yanchor="top", y=0.99, xanchor="right", x=1.1), height = 240, width = 800, margin={'t':20,'l':60,'b':40,'r':40})        
fig_starting_counts.show()

In [ ]:
fig_starting_counts.write_image('../../papers/this lab/mirror_repeats/plots/distributions_STRs_AnGn.pdf')